In [ ]:
import pandas as pd 
import os
import matplotlib.pyplot as plt

In [ ]:
#path = 'D:\\data_venv\\salesreport\\sales2019_1.csv'
#dataframe = pd.read_csv(path)
#dataframe.head(20)
path = 'D:\\data_venv\\salesreport\\'
os.listdir(path)

## TASK 2: CLEAN AND PROCESS DATA

### 2.1 MERGE 12-MONTH DATA

In [ ]:
## The data for each month was saved in separate files, and we need to reconcile them into one file
path = 'D:\\data_venv\\salesreport\\'
filepaths = []
frames = []

for file in os.listdir(path):
    if file.endswith('.csv'):
        filepath = os.path.join(path, file)  # Use os.path.join to create the full path
        filepaths.append(filepath)
        df1 = pd.read_csv(filepath)  # Read each CSV file into df1
        frames.append(df1)  # Append df1 to the frames list

result = pd.concat(frames, ignore_index=True)  # Concatenate all DataFrames in the frames list
result.to_csv('annualSales2019.csv', index=False)  # Save the concatenated DataFrame to a single CSV file
df = result

#### QUESTION 1: WHAT WAS THE BEST MONTH FOR SALES? HOW MUCH WAS EARNED THAT MONTH?

In [ ]:
# Add new column
df.loc[:, 'month'] = df['Order Date'].str.slice(0, 2)
# hoặc df['Order Date].str[0:2]
df.head(50)
print(set(df['month']))

### TASK1: GET RID OF NAN, OR VALUE

In [ ]:
df = df.dropna(how='all')
df.head()

In [ ]:
df = df[df['month'] != 'Or']
print(set(df['month']))

In [ ]:
print(df['Quantity Ordered'].dtypes)
print(df['Price Each'].dtypes)

TASK2: REPORTING

In [ ]:
# Convert 'Quantity Ordered' and 'Price Each' columns to numeric (if they are not already)
#df['Quantity Ordered'] = pd.to_numeric(df['Quantity Ordered'], errors='coerce')
#df['Price Each'] = pd.to_numeric(df['Price Each'], errors='coerce')

df['Quantity Ordered'] = pd.to_numeric(df['Quantity Ordered'], downcast = 'integer')
df['Price Each'] = pd.to_numeric(df['Price Each'], downcast = 'float')

# Create the 'Sales' column by multiplying the two numeric columns
df['Sales'] = df['Quantity Ordered'] * df['Price Each']

# Display the first 10 rows to check the result
df.head()

In [ ]:
column_to_move = df.pop("Sales")
# insert column with insert(location, column_name, column_value)
df.insert(4, "Sales", column_to_move)
df.head()

In [ ]:
# Group the DataFrame by the 'month' column and calculate the sum of 'Sales' in each group
monthly_revenue = df.groupby('month')['Sales'].sum()
print(monthly_revenue)

In [ ]:
# Calculate the 'Sales' column by multiplying 'Quantity Ordered' and 'Price Each'
df['Sales'] = df['Quantity Ordered'] * df['Price Each']

# Group the DataFrame by 'month' and calculate the sum of sales for each month
monthly_revenue = df.groupby('month')['Sales'].sum().reset_index()

# Find the row with the highest sales
highest_sales_row = monthly_revenue[monthly_revenue['Sales'] == monthly_revenue['Sales'].max()]

# Extract the month and sales value from the row
highest_sales_month_name = highest_sales_row['month'].values[0]
highest_sales_value = highest_sales_row['Sales'].values[0]

# Display the result
print(f"The month with the highest sales is {highest_sales_month_name} with total sales of ${highest_sales_value:.2f}")


In [ ]:
sales_value = df.groupby('month').sum()['Sales']
months = range(1,13)
plt.bar(x = months, height = sales_value)
plt.xticks(months)
plt.xlabel('Months')
plt.ylabel('Sales')
plt.show

##WHAT CITY HAS THE BEST REVENUE?

In [ ]:
df.head()

In [ ]:
address_to_city = lambda address:address.split(',')[1]

In [ ]:
df['City'] = df['Purchase Address'].apply(address_to_city)
df.head()

In [ ]:
df.groupby('City').sum()['Sales']

In [ ]:
sales_values_city = df.groupby('City').sum()['Sales']
sales_values_city.max()


In [ ]:
cities = []
for city, sales in sales_values_city.items():
    cities.append(city)
print(cities)
list(set(df['City']))  # Convert the set to a list
plt.bar(x=cities, height=sales_values_city)
plt.xlabel('City')
plt.xticks(rotation=45)
plt.ylabel('Sales')
plt.show()

#### TASK 3: WHAT PRODUCT SOLD THE MOST? WHY DO YOU THINK IT SOLD THE MOST?

In [ ]:
df['Order Date'] = pd.to_datetime(df['Order Date'], format='%Y-%m-%d %H:%M:%S')
print(df['Order Date'].dtypes)


In [ ]:
df['Hour'] = df['Order Date'].dt.hour
df.head()

In [ ]:
sales_values_hour = df.groupby('Hour')['Sales'].sum()
hours = [
hour for hour, sales in sales_values_hour.items()]
list(set(df['Hour']))  # Convert the set to a list
plt.plot(hours, sales_values_hour)
# Set the x-axis ticks to show every value of the hour
plt.xticks(hours)
plt.grid()
plt.xlabel('Hour')
plt.ylabel('Sales')
plt.show()

In [ ]:
sales_values_hour = df.groupby('Hour')['Sales'].count()
hours = [
hour for hour, sales in sales_values_hour.items()]
list(set(df['Hour']))  # Convert the set to a list
plt.plot(hours, sales_values_hour)
# Set the x-axis ticks to show every value of the hour
plt.xticks(hours)
plt.grid()
plt.xlabel('Hour')
plt.ylabel('Sales')
plt.show()

#### TASK 4: WHAT PRODUCT ARE MOST OFTEN SOLD TOGETHER?

In [ ]:
df_dup = df[df['Order ID'].duplicated(keep = False)]
df_dup.head()

In [ ]:
groupProduct = lambda product: ', '.join(product)

In [ ]:
# Use groupby and transform to create a new Series with aggregated 'Product' values
agg_products = df_dup.groupby('Order ID')['Product'].transform(groupProduct)

# Create a new DataFrame with the aggregated 'Product' values
df_modified = df_dup.copy()  # Create a copy of the original DataFrame
df_modified['All Products'] = agg_products
df_modified.head()

In [ ]:
df_dup2 = df_modified[['Order ID', 'All Products']].drop_duplicates()
df_dup2.head()

In [ ]:
df_dup2['All Products'].value_counts().head(10)

In [ ]:
products_sold = df.groupby('Product')['Quantity Ordered'].sum()
products_sold = products_sold.sort_values(ascending=False)
print(products_sold)

In [ ]:
# Convert the 'Price Each' column to strings
df['Price Each'] = df['Price Each'].astype(str)

# Remove non-numeric characters from the 'Price Each' column
df['Price Each'] = df['Price Each'].str.replace(r'[^0-9.]', '', regex=True)

# Convert the 'Price Each' column to a numeric data type
df['Price Each'] = pd.to_numeric(df['Price Each'])

# Calculate the mean of 'Price Each' for each product
prices = df.groupby('Product')['Price Each'].mean()
print(prices)

In [ ]:
product_ls = products_sold.index  # Get the product names from the index of products_sold
quantity_values = products_sold.values  # Get the quantity values

plt.bar(product_ls, quantity_values)
plt.xticks(product_ls, rotation=90, fontsize=8)
plt.xlabel('Products')
plt.ylabel('Quantity')
plt.show()

In [ ]:
x = product_ls
y1 = products_sold
y2 = prices

fig, ax1 = plt.subplots()

ax2 = ax1.twinx()
ax1.bar(x, y1, color='g')
ax2.plot(x, y2, 'b-')

ax1.set_xticklabels(product_ls, rotation = 90, size = 8)
ax1.set_xlabel('Product')  # Set x-axis label for the first axis
ax1.set_ylabel('Quantity', color='g')  # Set y-axis label for the first axis
ax2.set_ylabel('Price', color='b')  # Set y-axis label for the second axis

plt.show()